In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, neighbors,datasets
from lib.helpers import *
from sklearn import svm
from lib.cross_validations_lib import *
#import peakutils
import scipy.signal as signal
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift, estimate_bandwidth

In [3]:
def feature_extraction(signal,feature_dictionary,f_peak):
    feature=[]
    
    if(feature_dictionary["fft_peak_frequencies"]==1):
        feature=np.concatenate((feature,f_peak.reshape(f_peak.shape[1])), axis=0);
        
    if(feature_dictionary["mean"]==1):
        mean=np.mean(signal);
        feature=np.concatenate((feature,[mean]), axis=0)
    
    if(feature_dictionary["variance"]==1):
        variance=np.var(signal)
        feature=np.concatenate((feature,[variance]), axis=0)
    #crest factor 
    if(feature_dictionary["crest_factor"]==1):
        crest_factor=np.sum(np.power(signal,2))/signal.size
        feature=np.concatenate((feature,[crest_factor]), axis=0)
    
    return np.asarray(feature)
#.reshape([feature.size,1])


def split_matrix_two_blocks(y, percentage1, percentage2, seed):
    """Build k indices for k-fold."""
    if(percentage1+percentage2==1):
        num_row = len(y)
        print(num_row)
        interval_1 = int(percentage1*num_row);
        
        np.random.seed(seed)
        indices = np.random.permutation(num_row);
        first_indices = indices[0:interval_1];
        second_indices = indices[interval_1:num_row];
        return [np.array(first_indices),np.array(second_indices)]
    else:
        print('>>>>>>>>>>>ERROR:Not valid splitting percentage')
    


In [4]:
def export_signals(channels):
    time_instances=[];
    dim=channels.shape;
    #find the length min of the signal in the specified temporal instance
    # for NIRS signal ==80 
    length_min=len(channels[0,1]);
    for i in range (0,dim[1]):
        single_measurement=channels[0,i];
        single_length=single_measurement.shape[0]
        if(single_length<length_min):
                length_min=single_length;
    #export the signals
    for i in range (0,dim[1]):
        single_measurement=channels[0,i];
        dim1=single_measurement.shape;
        time_instance=[];
        for j  in range (0,dim1[1]):
            if(len(single_measurement[:,j])>length_min):
                single_signal=single_measurement[:,j][0:length_min]
            else:
                single_signal=single_measurement[:,j]
            #put in a list 
            time_instance.append(np.asarray(single_signal).reshape(len(single_signal),1).T);
       # create the matrix of the signals per a single time instance 
        time_instance=np.concatenate(time_instance);
        time_instances.append(time_instance);   
    return time_instances;






def get_feature_matrix_and_labels(channel_structure, feature_dictionary,label,features_extracted):
    list_train=[]
    list_labels=[]
    cont=0;
    peak_signal=features_extracted["peak_signal"]
    f_peak_signal=peak_signal["f_cell"];
    
    for time_instance in channel_structure:
        dim1=time_instance.shape
        for j  in range (0,dim1[0]):
            features=feature_extraction(time_instance[j,:],feature_dictionary,f_peak_signal[0,cont])
            list_train.append([features]);
            cont=cont+1;
        labels=get_labels(dim1[0],label);
        list_labels.append([labels]);
        
    train_TX=np.concatenate(list_train)
    labels=np.concatenate(list_labels,axis=1)
    
    return train_TX,labels.T.reshape(labels.size)


def get_feature_matrix(channels,feature_dictionary,features_extracted):
    list_train=[]
    list_labels=[]
    cont=0;
    peak_signal=features_extracted["peak_signal"]
    f_peak_signal=peak_signal["f_cell"];
    for time_instance in channel_structure:
        dim1=time_instace.shape
        print(dim1)
        for j  in range (0,dim1[0]):
            features=feature_extraction(time_instance[j,:],feature_dictionary,f_peak_signal[0,cont])
            list_train.append([features]);
            cont=cont+1;
    train_TX=np.concatenate(list_train)
    return train_TX

def get_labels(number, string):
    if(string=="No"):
        return np.zeros(number)    
    if(string=="Yes"):
        return np.ones(number)
    

# EEG 

In [5]:
yes_EEG_contents = sio.loadmat('EEGyes_signal.mat')
no_EEG_contents = sio.loadmat('EEGno_signal.mat')


channels_no_EEG=no_EEG_contents["no_signal"]
channels_yes_EEG=yes_EEG_contents["yes_signal"]

pick_yes_EEG_contents = sio.loadmat('pick_EEG_yes_signal.mat')
pick_no_EEG_contents = sio.loadmat('pick_EEG_no_signal.mat')

# select which feature select
feature_dictionary = {
        "fft_peak_frequencies" : 0, 
         "mean" : 0, 
         "variance" : 0,
         "crest_factor" : 0
         }




channels_structure_yes_EEG=export_signals(channels_yes_EEG)
channels_structure_no_EEG=export_signals(channels_no_EEG)


features_extracted={
    "peak_signal" : pick_yes_EEG_contents,
}


train_TX_yes_EEG,EEG_yes_labels=get_feature_matrix_and_labels(channels_structure_yes_EEG, feature_dictionary,"Yes",features_extracted);

features_extracted={
    "peak_signal" : pick_no_EEG_contents,
}


train_TX_no_EEG,EEG_no_labels=get_feature_matrix_and_labels(channels_structure_no_EEG, feature_dictionary,"No",features_extracted);



train_TX=np.concatenate((train_TX_yes_EEG,train_TX_no_EEG,),axis=0)

print(train_TX.shape)
labels=np.concatenate((EEG_yes_labels,EEG_no_labels),axis=0)



(780, 6)


In [ ]:
C_parameters= np.linspace(0.1,5,10)
k_fold=5 # number of k sub-folders to divide the set
seed = 2
max_iters = 100
kernel_types=['linear', 'rbf', 'sigmoid']
print(train_TX.shape)

best_C, best_kernel_type ,best_accuracy_test, corresponding_accuracy_train = \
        cross_validation_SVM(labels,train_TX, C_parameters, kernel_types, k_fold, seed, max_iters)



    

In [26]:
best_C=0.1;
best_kernel_type='linear'
print(train_TX.shape[0])

780


In [31]:
from sklearn import svm
best_C=0.1;
seed=[123,12,3,4,2,1]
best_kernel_type='linear'
dataset_length=train_TX.shape[0];

[i1,i2]=split_matrix_two_blocks(train_TX,0.7,0.3,2)
train=train_TX[i1,:]
labels_train=labels[i1]
test= train_TX[i2,:]
labels_test=labels[i2]
clf = svm.SVC(C=best_C, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovr', degree=3, gamma='auto', kernel=best_kernel_type,
        max_iter=-1, probability=False, random_state=None, shrinking=True,
        tol=0.001, verbose=False)


clf.fit(train, labels_train)  
predicted_labels= clf.predict(test)


SVM_accuracy=get_accuracy(predicted_labels, labels_test)
print(SVM_accuracy)
    

780
0.474358974359
